# PADL: G code generation tool

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;"> This tool is specifically created to generate G-code for the customised 3D printer device PADL, tailored for use with well plates.

## 0. Setup

> This section includes packages required to download.

<div class="alert alert-info">
    <b>Step 0:</b> Please download the following packages if you haven't already.

</div>

### Installation notes

To run this notebook you will need to install several packages, including Voila and Pygcode.

1. Voila can be installed via Conda:  __[linked reference](https://voila.readthedocs.io/en/stable/install.html#install)__

```bash
conda install -c conda-forge voila
```    
or with pip:

```bash
pip install voila
```

2. Python-Gcode can be installed via pip only:  __[linked reference](https://pypi.org/project/pygcode/)__

```bash
pip install pygcode
```

3. Python widgets can be installed via pip only:  __[linked reference](https://pypi.org/project/ipywidgets/)__

```bash
pip install ipywidgets
```

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;"> 

Congratulations! You have successfully downloaded everything. 

Please proceed by going to the <strong>Menu</strong>, selecting <strong> Kernel</strong>, <strong>Restart & Run all</strong>. 

After that, click on the Voila button at the top to initiate the G-code generation process!

</div>


In [1]:
import pygcode
import ipywidgets
import time

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import json

## 1. Initialise input Coordinates

> This section involves initializing the well plate and setting up the initial coordinates.

<div class="alert alert-info">
    <b>Step 1:</b> Please choose the number of <b>rows, columns, X step size, and Y step size</b> for your well plate.

</div>

<div class="alert alert-block alert-warning">
<b>Example:</b> 
The default values are the first four lines of the saved data <b>well_plate.json</b>, which start with lowercase letters.
    
For example, consider a <b>96-well plate</b> with rows <b>labeled A to H</b> and columns <b>numbered 1 to 12</b>, with a 9mm gap between each well.
    
        'rows': 8,
        'columns': 12,
        'x step size': 9.0,
        'y step size': -9.0,
</div>

In [2]:
# Define the integer widgets
rows = widgets.IntText(description='Rows')
columns = widgets.IntText(description='Columns')
x_step_size = widgets.FloatText(description='X step size')
y_step_size = widgets.FloatText(description='Y step size')

# Display the integer widgets
display(rows)
display(columns)
display(x_step_size)
display(y_step_size)

# Load data from a JSON file if it exists, otherwise initialize
try:
    with open('well_plate.json', 'r') as file0:
        well_plate = json.load(file0)
except FileNotFoundError:
    well_plate = {
        'rows': 8,
        'columns': 12,
        'x step size': 9.0,
        'y step size': -9.0,
        '':'',
        'Rows': 8,
        'Columns': 12,
        'X step size': 9.0,
        'Y step size': -9.0,
    }

# Define the output widget for integer values
intout_text = widgets.Textarea(
    value='',  # Initialize with an empty string
    description='Wellplate Setup:',
    disabled=False,
    layout=widgets.Layout(height='200px', width='350px')
)

# Define the function to update integer values
def update_values(change):
    variable_name = change['owner'].description
    well_plate[variable_name] = change['new']  # Replace the existing value
    
    # Update the output text
    intout_text.value = "\n".join([f"{key}: {value}" for key, value in well_plate.items()])
    
    # Save the updated data to the JSON file
    with open('well_plate.json', 'w') as file:
        json.dump(well_plate, file, indent=4)

# Observe changes in the integer widgets
rows.observe(update_values, names='value')
columns.observe(update_values, names='value')
x_step_size.observe(update_values, names='value')
y_step_size.observe(update_values, names='value')

# Display the output widget
display(intout_text)

# Display initial values
intout_text.value = "\n".join([f"{key}: {value}" for key, value in well_plate.items()]) 

# Assign the loaded integer values to variables
x_step_size = well_plate['X step size']
y_step_size = well_plate['Y step size']
rows = well_plate['Rows']
columns = well_plate['Columns']

IntText(value=0, description='Rows')

IntText(value=0, description='Columns')

FloatText(value=0.0, description='X step size')

FloatText(value=0.0, description='Y step size')

Textarea(value='', description='Wellplate Setup:', layout=Layout(height='200px', width='350px'))

<div class="alert alert-info">
    <b>Step 2:</b> Please choose the coordinates of <b>reservoir, water, head_away, and initial_A1</b> for your well plate.

</div>

<div class="alert alert-block alert-warning">
<b>Example:</b> 
The default values are the first four lines of the saved data <b>option_value.json</b>, which start with lowercase letters.
    
Sample coordinates:
    
        reservoir: X150.0 Y204.0 Z60.0
        water: X150.0 Y204.0 Z5.0
        head_away: X150.0 Y204.0 Z60.0
        initial_A1: X63.0 Y156.0 Z6
</div>

In [3]:
X_text = widgets.FloatText(description='X:')
Y_text = widgets.FloatText(description='Y:')
Z_text = widgets.FloatText(description='Z:')

print('Please choose your coordinates first and variable name second')
print('Suggest that ')
display(X_text)
display(Y_text)
display(Z_text)

number_dropdown = widgets.Dropdown(
    options=[('', 'Select an option'), ('Reservoir', 'Reservoir'), ('Water', 'Water'), ('Head_away', 'Head_away'), 
             ('A1', 'A1')],
    description='For variable'
)
display(number_dropdown)

# Load data from a JSON file if it exists, otherwise initialize
try:
    with open('option_values.json', 'r') as file:
        option_values = json.load(file)
except FileNotFoundError:
    option_values = {
        'reservoir': 'X150.0 Y204.0 Z60.0',
        'water': 'X150.0 Y204.0 Z5.0',
        'head_away': 'X150.0 Y204.0 Z60.0',
        'initial_A1': 'X63.0 Y156.0 Z6',
        '':'',
        'Reservoir': 'X150.0 Y204.0 Z60.0',
        'Water': 'X150.0 Y204.0 Z5.0',
        'Head_away': 'X150.0 Y204.0 Z60.0',
        'A1': 'X63.0 Y156.0 Z6',
    }

output_text = widgets.Textarea(
    value="",  # Initialize with an empty string
    description='Coordinates of variables:',
    disabled=False,
    layout=widgets.Layout(height='200px', width='350px')
)

def update_values(change):
    selected_option = change['new']
    option_values[selected_option] = f'X{X_text.value} Y{Y_text.value} Z{Z_text.value}'
    output_text.value = "\n".join([f"{key}: {value}" for key, value in option_values.items()])
    # Save the updated data to the JSON file
    with open('option_values.json', 'w') as file:
        json.dump(option_values, file, indent=4)

number_dropdown.observe(update_values, names='value')

display(output_text)

# Display initial values
output_text.value = "\n".join([f"{key}: {value}" for key, value in option_values.items()]) 

reservoir = option_values['Reservoir']
water = option_values['Water']
head_away = option_values['Head_away']
initial_A1 = option_values['A1']

Please choose your coordinates first and variable name second
Suggest that 


FloatText(value=0.0, description='X:')

FloatText(value=0.0, description='Y:')

FloatText(value=0.0, description='Z:')

Dropdown(description='For variable', options=(('', 'Select an option'), ('Reservoir', 'Reservoir'), ('Water', …

Textarea(value='', description='Coordinates of variables:', layout=Layout(height='200px', width='350px'))

## 2. Speed settings

> This section include selecting the moving speed of the extruder.

<div class="alert alert-info">
    <b>Step 3:</b> Please choose the <b>extruder's moving speed</b> for navigating to the coordinates of the well plate cells.

</div>

<div class="alert alert-block alert-warning">
<b>Example:</b> 
    
        
Recommended speed:
    
        move_speed: ' F3000'
        well_speed: ' F2000'     
        fill_tube_speed: ' F10'    
        aspirate_speed: ' F20' 
</div>

In [4]:
# Define the integer widgets
move_speed_widget = widgets.IntText(description='Move_speed')
well_speed_widget = widgets.IntText(description='Well_speed')
dispense_speed_widget = widgets.IntText(description='Dispense_speed')
aspirate_speed_widget = widgets.IntText(description='Aspirate_speed')
wait_between_cells_sec_widget = widgets.FloatText(description='Wait_between_cells_sec')
wait_between_experiment_sec_widget = widgets.FloatText(description='Wait_between_experiment_sec')
wait_initial_widget = widgets.FloatText(description='Wait_initial')

# Display the integer widgets
display(move_speed_widget)
display(well_speed_widget)
display(dispense_speed_widget)
display(aspirate_speed_widget)
display(wait_between_cells_sec_widget)
display(wait_between_experiment_sec_widget)
display(wait_initial_widget)

# Load data from a JSON file if it exists, otherwise initialize
try:
    with open('speed.json', 'r') as file2:
        speed = json.load(file2)
except FileNotFoundError:
    speed = {
        'move_speed': 3000,
        'well_speed': 2000,     
        'dispense_speed': 5,   
        'aspirate_speed': 20, 
        'wait_initial': 120,
        'wait_between_cells_sec': 0.2,
        'wait_between_experiment_sec': 3600,
        '':'',
        'Move_speed': 3000,
        'Well_speed': 2000,     
        'Dispense_speed': 5,   
        'Aspirate_speed': 20,
        'Wait_initial': 120,
        'Wait_between_cells_sec': 0.2,
        'Wait_between_experiment_sec': 3600,
    }

# Define the output widget for integer values
set_text = widgets.Textarea(
    value='',  # Initialize with an empty string
    description='Speed Setup:',
    disabled=False,
    layout=widgets.Layout(height='350px', width='350px')
)

# Define the function to update integer values
def update_values(change):
    variable_name = change['owner'].description
    speed[variable_name] = change['new']  # Replace the existing value
    
    # Update the output text
    set_text.value = "\n".join([f"{key}: {value}" for key, value in speed.items()])
    
    # Save the updated data to the JSON file
    with open('speed.json', 'w') as file:
        json.dump(speed, file, indent=4)

# Observe changes in the integer widgets
move_speed_widget.observe(update_values, names='value')
well_speed_widget.observe(update_values, names='value')
dispense_speed_widget.observe(update_values, names='value')
aspirate_speed_widget.observe(update_values, names='value')
wait_between_cells_sec_widget.observe(update_values, names='value')
wait_between_experiment_sec_widget.observe(update_values, names='value')
wait_initial_widget.observe(update_values, names='value')

# Display the output widget
display(set_text)

# Display initial values
set_text.value = "\n".join([f"{key}: {value}" for key, value in speed.items()]) 

# Assign the loaded integer values to variables with 'F' prefix
dispense_speed = ' F' + str(speed['Dispense_speed'])
aspirate_speed = ' F' + str(speed['Aspirate_speed'])
move_speed = ' F' + str(speed['Move_speed'])
well_speed = ' F' + str(speed['Well_speed'])
wait_initial = 'S' + str(speed['Wait_initial']) 
wait_between_cells_sec = 'S' + str(speed['Wait_between_cells_sec']) 
wait_between_experiment_sec = 'S' + str(speed['Wait_between_experiment_sec']) 


IntText(value=0, description='Move_speed')

IntText(value=0, description='Well_speed')

IntText(value=0, description='Dispense_speed')

IntText(value=0, description='Aspirate_speed')

FloatText(value=0.0, description='Wait_between_cells_sec')

FloatText(value=0.0, description='Wait_between_experiment_sec')

FloatText(value=0.0, description='Wait_initial')

Textarea(value='', description='Speed Setup:', layout=Layout(height='350px', width='350px'))

## 3. Get the well-plate coordinates

> This section involves generating the well-plate coordinates based on the initialisation step that precedes it.

<div class="alert alert-info">
    <b>Step 4:</b> Please verify the coordinates for H12 with the 3D printer coordinates to ensure they match. If they do not match, it may be necessary to update the X and Y coordinates accordingly.
</div>

In [5]:
# Define your 3D printer settings
fill_tube_speed = ' F10'

Move = 'G0 '
Liquid = 'G1 '
Stop = 'G4 '

v3 = 'E4'
v_1 = 'E-1'
v_dispense = 'E-0.1'

In [6]:
def generate_sequence_y(initial, y_step_size, num_value):
    sequence = []
    for i in range(num_value):
        point = generate_next_point_y(initial, y_step_size * i)
        sequence.append(point)
    return sequence

def generate_sequence_x(initial, x_step_size, num_value):
    sequence = []
    for i in range(num_value):
        point = generate_next_point_x(initial, x_step_size * i)
        sequence.append(point)
    return sequence

def generate_next_point_y(current_point, y_step_size):
    parts = current_point.split()
    if len(parts) != 3 or not parts[0].startswith('X') or not parts[1].startswith('Y') or not parts[2].startswith('Z'):
        raise ValueError("Input format should be 'X## Y## Z##'")
    
    x_val = parts[0]
    z_val = parts[2]
    y_val = float(parts[1][1:]) + y_step_size
    
    next_point = f'{x_val} Y{y_val} {z_val}'
    return next_point

def generate_next_point_x(current_point, x_step_size):
    parts = current_point.split()
    if len(parts) != 3 or not parts[0].startswith('X') or not parts[1].startswith('Y') or not parts[2].startswith('Z'):
        raise ValueError("Input format should be 'X## Y## Z##'")
    
    x_val = float(parts[0][1:]) + x_step_size
    y_val = parts[1]
    z_val = parts[2]
    
    next_point = f'X{x_val} {y_val} {z_val}'
    return next_point

In [7]:
V1 = generate_sequence_y(initial_A1, y_step_size, rows)

coordinate_mapping = {}
step = 1

for y in range(rows):
    row_label = chr(ord('A') + y)  # Convert index to row label

    x_sequence = generate_sequence_x(V1[y], x_step_size, columns)

    for x, value in enumerate(x_sequence):
        column_label = str(x + 1)  # Convert index to column label
        well_label = row_label + column_label
        coordinate_mapping[well_label] = value

# Display coordinate_mapping in a more readable format
for well_label, value in coordinate_mapping.items():
    print(f'{well_label}: {value}')

A1: X63.0 Y156.0 Z6
A2: X72.0 Y156.0 Z6
A3: X81.0 Y156.0 Z6
A4: X90.0 Y156.0 Z6
A5: X99.0 Y156.0 Z6
A6: X108.0 Y156.0 Z6
A7: X117.0 Y156.0 Z6
A8: X126.0 Y156.0 Z6
A9: X135.0 Y156.0 Z6
A10: X144.0 Y156.0 Z6
A11: X153.0 Y156.0 Z6
A12: X162.0 Y156.0 Z6
B1: X63.0 Y147.0 Z6
B2: X72.0 Y147.0 Z6
B3: X81.0 Y147.0 Z6
B4: X90.0 Y147.0 Z6
B5: X99.0 Y147.0 Z6
B6: X108.0 Y147.0 Z6
B7: X117.0 Y147.0 Z6
B8: X126.0 Y147.0 Z6
B9: X135.0 Y147.0 Z6
B10: X144.0 Y147.0 Z6
B11: X153.0 Y147.0 Z6
B12: X162.0 Y147.0 Z6
C1: X63.0 Y138.0 Z6
C2: X72.0 Y138.0 Z6
C3: X81.0 Y138.0 Z6
C4: X90.0 Y138.0 Z6
C5: X99.0 Y138.0 Z6
C6: X108.0 Y138.0 Z6
C7: X117.0 Y138.0 Z6
C8: X126.0 Y138.0 Z6
C9: X135.0 Y138.0 Z6
C10: X144.0 Y138.0 Z6
C11: X153.0 Y138.0 Z6
C12: X162.0 Y138.0 Z6
D1: X63.0 Y129.0 Z6
D2: X72.0 Y129.0 Z6
D3: X81.0 Y129.0 Z6
D4: X90.0 Y129.0 Z6
D5: X99.0 Y129.0 Z6
D6: X108.0 Y129.0 Z6
D7: X117.0 Y129.0 Z6
D8: X126.0 Y129.0 Z6
D9: X135.0 Y129.0 Z6
D10: X144.0 Y129.0 Z6
D11: X153.0 Y129.0 Z6
D12: X162.0 Y129.0 Z6


## 4. Establishing G-Code Commands for Sample Setup

> This section includes selecting the well plate cells and providing them as the output.

<div class="alert alert-info">
    <b>Step 5:</b> Please select the coordniates chronologically
</div>

In [8]:
# Create an empty list to store G-code commands
gcode_commands = []

gcode_commands.append('; Initialise G code iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii')
# Disable cold extrusion checking
gcode_commands.append('M302 P1')

# Set steps-per-ml for 5ml syringe
gcode_commands.append('M92 E34008')

# Absolute positioning
gcode_commands.append('G90')

# Set syringe pump to relative positioning
gcode_commands.append('M83')

# Home all axes
gcode_commands.append('G28')

gcode_commands.append('G90')

# Move to the initial position
gcode_commands.append('G0 X0 Y0 Z35 F5000')

In [9]:
# Get water and Fill resevior 
components = [reservoir, water]

for component in components:
    gcode_commands.append(Move + component + move_speed)

gcode_commands.append('')
gcode_commands.append(Liquid + v3 + aspirate_speed)
gcode_commands.append(Move + reservoir + move_speed)
gcode_commands.append('M83')
gcode_commands.append(Liquid + v_1 + fill_tube_speed)

components = [reservoir, head_away]

for component in components:
    gcode_commands.append(Move + component + move_speed)

In [10]:
# Create an initial output_textarea with a fixed height
output_textarea = widgets.Textarea(value='', description='Output List:', layout=widgets.Layout(height='100px', width='400px'))

# Function to dynamically adjust the textarea height based on content
def adjust_textarea_height(change):
    lines = change.new.split('\n')
    num_lines = len(lines)
    new_height = max(100, num_lines * 20)  # Minimum height of 300px

    # Update the height of the textarea
    output_textarea.layout.height = f'{new_height}px'

# Attach the event handler to the output_textarea
output_textarea.observe(adjust_textarea_height, 'value')

In [11]:
Wellplate = []
output = widgets.Output()
output_list = []  # List to store button descriptions
last_click_time = {}  # Dictionary to store the last click time for each button
prev_clicked_button = None  # Keep track of the previously clicked button

def on_button_clicked(b):
    global prev_clicked_button, output_list
    
    current_time = time.time()
    last_time = last_click_time.get(b, 0)
    
    if current_time - last_time < 0.5:  # Check if it's a double-click
        print("Double-click detected. Cancelling click action.")
        last_click_time[b] = 0  # Reset last click time
        b.style.button_color = 'transparent'  # Set button color
        if str(b.description) in output_list:
            output_list.remove(str(b.description))  # Remove description from the list
            
            if prev_clicked_button and str(prev_clicked_button.description) in output_list:
                output_list.remove(str(prev_clicked_button.description))  # Remove previous button from the list
                
            prev_clicked_button = None  # Reset the previously clicked button
        return
    
    last_click_time[b] = current_time  # Store current click time
    prev_clicked_button = b
    
    with output:
        print(str(b.description))
        b.style.button_color = 'lightgreen'
        output_list.append(str(b.description))  # Append description to the list
        output_textarea.value = '\n'.join(output_list)  # Update the textarea widget
        # print("Current List:", output_list)  # Display updated list

# Generate the grid for the 96 well-plate
allrows = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
buttons_grid = []

for row in allrows[0:rows]:
    row_buttons = []
    for i in range(1, columns+1):
        button = widgets.Button(description=row + str(i))
        button.on_click(on_button_clicked)
        row_buttons.append(button)
        Wellplate.append(button.description)
        last_click_time[button] = 0  # Initialize last click time
    buttons_grid.append(widgets.HBox(row_buttons))

button_grid_box = widgets.VBox(buttons_grid)

display(button_grid_box, output_textarea)


Textarea(value='', description='Output List:', layout=Layout(height='100px', width='400px'))

In [12]:
def update_coordinates(label_list, coordinate_mapping):
    updated_list = []
    
    for label in label_list:
        if label in coordinate_mapping:
            updated_list.append(coordinate_mapping[label])
        else:
            updated_list.append(label)
    
    return updated_list

gcode_commands.append('; Start of the experimenttttttttttttttttttttttttttttttttttttttttttttttttttttttt')
gcode_commands.append('M83')
gcode_commands.append(Stop + str(wait_initial))
gcode_commands.append('')

## 6. Save Python G-Code and Export as a .gcode File

> This section encompasses displaying G-code commands and exporting them.

<div class="alert alert-info">
    <b>Step 7:</b> Please input a name and proceed to export the G-code files.
</div>

In [13]:
# Create a text input widget
text_input = widgets.Text(
    value='',
    description='Saved as:',
    placeholder='Type your name here',
    disabled=False
)

# Display the widgets
display(text_input)

Text(value='', description='Saved as:', placeholder='Type your name here')

In [14]:
button_send = widgets.Button(
                description='Send to PADL G code generation tool',
                tooltip='Send',
                style={'description_width': 'initial'}
            )

output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        print(f"Messages saved to {text_input.value}.gcode")
        print("Sent message (This is what your gcode looks like):  ")
        print("\n")

        for component in update_coordinates(output_list, coordinate_mapping):
            gcode_commands.append(Move + component + well_speed)
            gcode_commands.append(';' + str(component))
            gcode_commands.append(Liquid + v_dispense + dispense_speed)
            gcode_commands.append(Stop + wait_between_cells_sec)
        
        gcode_commands.append('')
        gcode_commands.append(Stop + str(wait_between_experiment_sec))
        gcode_commands.append('')
        
        for command in gcode_commands:
            print(command)
        
        filename = text_input.value+'.gcode'
        # Save the G-code commands to a .gcode file
        with open(filename, 'w') as f:
            for command in gcode_commands:
                f.write(command + '\n')
            
button_send.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_send, output])

In [15]:
# Existing code for the "Send to PADL G code generation tool" button and output widget
button_send = widgets.Button(
    description='Send to PADL G code generation tool',
    tooltip='Send',
    style={'description_width': 'initial'}
)

button_tool_change = widgets.Button(
    description='Tool Change',
    tooltip='Perform Tool Change',
    style={'description_width': 'initial'}
)

output = widgets.Output()

def on_button_clicked(event):
    # Function to handle the "Send to PADL G code generation tool" button click
    with output:
        clear_output()
        print(f"Messages saved to {text_input.value}.gcode")
        print("Sent message (This is what your gcode looks like):  ")
        print("\n")

        for component in update_coordinates(output_list, coordinate_mapping):
            gcode_commands.append(Move + component + well_speed)
            gcode_commands.append(Liquid + v_dispense + dispense_speed)
            gcode_commands.append(Stop + wait_between_cells_sec)
        
        gcode_commands.append('')
        gcode_commands.append('; Waiting time ..............................................................')
        gcode_commands.append(Stop + str(wait_between_experiment_sec))
        gcode_commands.append('')
        
        for command in gcode_commands:
            print(command)
        
        filename = text_input.value+'.gcode'
        # Save the G-code commands to a .gcode file
        with open(filename, 'w') as f:
            for command in gcode_commands:
                f.write(command + '\n')

def on_tool_change_clicked(event):
    # Function to handle the "Tool Change" button click
    # initialise 3d printer
    gcode_commands.append('; Tool changing module AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA')
    gcode_commands.append('G0 A F500') # sets A rotation to 500 deg/min
    gcode_commands.append('G0 A8.3 F500') # ensures home positioning of frame mount
    gcode_commands.append('G28') # autohome
    
    # Docking to frame code:
    gcode_commands.append('G0 X242 Y0 Z115 F5000') # Docking position to frame
    
    # Moving away
    gcode_commands.append('G0 X200')
    gcode_commands.append('G4 S10')
    
    # Detatching from frame code:
    gcode_commands.append('G0 X240 Z115')
    gcode_commands.append('G0 A3 F500') # detatch from frame mount
    gcode_commands.append('G0 X150') # move away with tool
    gcode_commands.append('G0 A8.3') # Reset positioning of frame mount
    gcode_commands.append('; Tool changing module end DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD')
    gcode_commands.append('')
    
button_send.on_click(on_button_clicked)
button_tool_change.on_click(on_tool_change_clicked)

vbox_result = widgets.VBox([button_send, button_tool_change, output])


In [16]:
page = widgets.HBox([vbox_result])
display(page)